In [5]:
print("hello")

hello


In [6]:
import os
os.chdir("/Users/laruelinder/MedicalChatbot")
%pwd

'/Users/laruelinder/MedicalChatbot'

In [7]:

from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [ ]:
#loads the pdf file using langhchahain pdf and directory loaders 
#returns a list of document objects 
#one document object per page stored in a list. each object has metadata (source and pg#) and page content
def load_pdf_file(data):
    loader = DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents




In [9]:
extracted_data = load_pdf_file("Data/")

In [ ]:
#extracted_data

[Document(metadata={'source': 'Data/GaleMedAC.pdf', 'page': 0}, page_content='The GALE\nENCYCLOPEDIA of\nAlTERNATIVE\nMEDICINE\nSECOND EDITION\nGEAMv1htp/tp  10/11/04  10:44 AM  Page 1'),
 Document(metadata={'source': 'Data/GaleMedAC.pdf', 'page': 1}, page_content='The GALE\nENCYCLOPEDIA of\nAlternative\nMEDICINE\nSECOND EDITION\nJACQUELINE L. LONGE, PROJECT EDITOR\nVOLUME\nA-C\n1\nGEAMv1htp/tp  10/11/04  10:44 AM  Page 3'),
 Document(metadata={'source': 'Data/GaleMedAC.pdf', 'page': 2}, page_content="The Gale Encyclopedia of Alternative Medicine, Second Edition\nProject Editor\nJacqueline L. Longe\nEditorial\nDeirdre S. Blanchfield, Laurie Fundukian, Erin\nWatts\nEditorial Support Services\nAndrea Lopeman\nRights Acquisition Management\nMargaret Abendroth, Ann Taylor\nImaging\nRandy Bassett, Lezlie Light, Dan W. Newell,\nRobyn V. Young\nProduct Design\nMichelle DiMercurio, Tracey Rowens\nComposition and Electronic Prepress\nEvi Seoud, Mary Beth Trimper\nManufacturing\nWendy Blurton, D

In [ ]:
#split the text into chunks
#returns a list of documents where each document is a chunk based on the page_content
def split_text(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
    #split at the document level
    text_chunks= text_splitter.split_documents(extracted_data)
    return text_chunks


In [ ]:
#text chunks is a list of broken up documents
text_chunks = split_text(extracted_data)
text_chunks

[Document(metadata={'source': 'Data/GaleMedAC.pdf', 'page': 0}, page_content='The GALE\nENCYCLOPEDIA of\nAlTERNATIVE\nMEDICINE\nSECOND EDITION\nGEAMv1htp/tp  10/11/04  10:44 AM  Page 1'),
 Document(metadata={'source': 'Data/GaleMedAC.pdf', 'page': 1}, page_content='The GALE\nENCYCLOPEDIA of\nAlternative\nMEDICINE\nSECOND EDITION\nJACQUELINE L. LONGE, PROJECT EDITOR\nVOLUME\nA-C\n1\nGEAMv1htp/tp  10/11/04  10:44 AM  Page 3'),
 Document(metadata={'source': 'Data/GaleMedAC.pdf', 'page': 2}, page_content='The Gale Encyclopedia of Alternative Medicine, Second Edition\nProject Editor\nJacqueline L. Longe\nEditorial\nDeirdre S. Blanchfield, Laurie Fundukian, Erin\nWatts\nEditorial Support Services\nAndrea Lopeman\nRights Acquisition Management\nMargaret Abendroth, Ann Taylor\nImaging\nRandy Bassett, Lezlie Light, Dan W. Newell,\nRobyn V. Young\nProduct Design\nMichelle DiMercurio, Tracey Rowens\nComposition and Electronic Prepress\nEvi Seoud, Mary Beth Trimper\nManufacturing\nWendy Blurton, D

In [16]:
from langchain.embeddings import HuggingFaceEmbeddings

def download_huggingface_embeddings():
    #hugging face embedding model returns 384 dimensional vector
    return HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [18]:
embeddings = download_huggingface_embeddings()


In [19]:
query_results = embeddings.embed_query("hello, world")
print("Length of query results: ", len(query_results))
print("Query results: ", query_results)

Length of query results:  384
Query results:  [-0.05609436333179474, 0.03554251790046692, 0.004592809826135635, 0.02386581152677536, -0.04944375529885292, -0.15514135360717773, 0.065925732254982, 0.022491345182061195, -0.02172732539474964, 0.01411921065300703, 0.05505155026912689, 0.024055682122707367, 0.0050191800110042095, -0.006477487739175558, -0.03411556035280228, -0.05552099272608757, -0.006752630230039358, -0.023014020174741745, -0.17627854645252228, -0.023092038929462433, 1.4229985026759095e-05, 0.07931121438741684, -0.012627890333533287, 0.037130068987607956, -0.09230007231235504, -0.023067859932780266, 0.060699619352817535, 0.0513303168118, -0.029477503150701523, -0.03724541515111923, 0.037288516759872437, 0.051599614322185516, 0.09633669257164001, -0.00937416311353445, -0.013310346752405167, 0.0866255834698677, -0.08137475699186325, -0.06393172591924667, 0.005632359534502029, 0.018668079748749733, 0.05008465424180031, -0.07173820585012436, -0.055224623531103134, -0.046738840

In [20]:
from dotenv import load_dotenv
load_dotenv()

True

In [25]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

In [28]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalchatbot"

pc.create_index(
    name=index_name,
    dimension=384, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)



PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': '11cc1c21711714bf20317754ad8cf3e0', 'Date': 'Tue, 07 Jan 2025 23:16:48 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [32]:
import os 
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

In [ ]:
from langchain_pinecone import PineconeVectorStore  
 #embed each chunk and upsert into pinecone index  
docsearch = PineconeVectorStore.from_documents(
    documents = text_chunks, 
    index_name=index_name, 
    embedding = embeddings
)

